In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [3]:
url_tab = 'https://www.resetera.com/threads/media-create-sales-week-53-2020-dec-28-jan-03.356956/'


In [4]:
ustitles = requests.get(url_tab)
soup = BeautifulSoup(ustitles.text)
aa=soup.findAll("div", {"class": "bbWrapper"})
print(aa[0].text)

Media Create Sales: Week 53, 2020 (Dec 28 - Jan 03)

01./01. [NSW] Momotaro Dentetsu: Showa, Heisei, Reiwa mo Teiban! <TBL> (Konami) {2020.11.19} (¥6.300)
02./02. [NSW] Animal Crossing: New Horizons # <ETC> (Nintendo) {2020.03.20} (¥5.980)
03./03. [NSW] Ring Fit Adventure # <HOB> (Nintendo) {2019.10.18} (¥7.980)
04./04. [NSW] Mario Kart 8 Deluxe <RCE> (Nintendo) {2017.04.28} (¥5.980)
05./05. [NSW] Super Smash Bros. Ultimate # <FTG> (Nintendo) {2018.12.07} (¥7.200)

Top 5

NSW - 5

(Taiwan)
01./01. [NSW] Ring Fit Adventure <HOB> (Nintendo) {2019.10.31}
02./04. [NSW] Mario Kart 8 Deluxe <RCE> (Nintendo) {2017.12.15}
03./05. [NSW] Super Mario Party <ETC> (Nintendo) {2018.10.05}
04./00. [NSW] Animal Crossing: New Horizons <ETC> (Nintendo) {2020.03.20}
05./02. [NSW] Hyrule Warriors: Age of Calamity <ACT> (Nintendo) {2020.11.20}

Top 5

NSW - 5

(South Korea)
01./02. [NSW] Ring Fit Adventure <HOB> (Nintendo) {2019.10.18}
02./01. [NSW] Animal Crossing: New Horizons <ETC> (Nintendo) {2020.03.2

In [5]:
tab_top=aa[0].text.split("Famitsu Sales: Week")[1]
#tab_top=tab_top.replace("[","###").replace("]","###").replace("<","###").replace("./","###")
#tab_top=tab_top.replace(">","/").replace("(","###").replace(")","###").replace("{","###").replace("}","###")
tab_top=tab_top.replace(" # ","").split("\n")
#re.split('; |, |\*|\n',a)
tab_top[2:32]

['01./01. [NSW] Momotaro Dentetsu: Showa, Heisei, Reiwa mo Teiban! <TBL> (Konami) {2020.11.19} (¥6.300) - 190.427 / 1.423.450 <80-100%> (-29%)',
 '02./02. [NSW] Animal Crossing: New Horizons<ETC> (Nintendo) {2020.03.20} (¥5.980) - 94.382 / 6.472.485 <80-100%> (-12%)',
 '03./03. [NSW] Mario Kart 8 Deluxe <RCE> (Nintendo) {2017.04.28} (¥5.980) - 83.853 / 3.541.036 <80-100%> (+23%)',
 '04./04. [NSW] Ring Fit Adventure<HOB> (Nintendo) {2019.10.18} (¥7.980) - 70.714 / 2.157.719 <80-100%> (+21%)',
 '05./06. [NSW] Super Smash Bros. Ultimate<FTG> (Nintendo) {2018.12.07} (¥7.200) - 54.865 / 4.068.039 <80-100%> (+3%)',
 '06./09. [NSW] Clubhouse Games: 51 Worldwide Classics <TBL> (Nintendo) {2020.06.05} (¥3.980) - 45.354 / 565.003 <80-100%> (+14%)',
 '07./08. [NSW] Super Mario Party<ETC> (Nintendo) {2018.10.05} (¥5.980) - 44.867 / 1.774.663 <80-100%> (+7%)',
 '08./07. [NSW] Minecraft<ADV> (Microsoft Game Studios) {2018.06.21} (¥3.600) - 37.530 / 1.740.451 <80-100%> (-23%)',
 '09./05. [NSW] Pikmin

In [6]:
clean_tab=tab_top[2:32]
clean_tab

['01./01. [NSW] Momotaro Dentetsu: Showa, Heisei, Reiwa mo Teiban! <TBL> (Konami) {2020.11.19} (¥6.300) - 190.427 / 1.423.450 <80-100%> (-29%)',
 '02./02. [NSW] Animal Crossing: New Horizons<ETC> (Nintendo) {2020.03.20} (¥5.980) - 94.382 / 6.472.485 <80-100%> (-12%)',
 '03./03. [NSW] Mario Kart 8 Deluxe <RCE> (Nintendo) {2017.04.28} (¥5.980) - 83.853 / 3.541.036 <80-100%> (+23%)',
 '04./04. [NSW] Ring Fit Adventure<HOB> (Nintendo) {2019.10.18} (¥7.980) - 70.714 / 2.157.719 <80-100%> (+21%)',
 '05./06. [NSW] Super Smash Bros. Ultimate<FTG> (Nintendo) {2018.12.07} (¥7.200) - 54.865 / 4.068.039 <80-100%> (+3%)',
 '06./09. [NSW] Clubhouse Games: 51 Worldwide Classics <TBL> (Nintendo) {2020.06.05} (¥3.980) - 45.354 / 565.003 <80-100%> (+14%)',
 '07./08. [NSW] Super Mario Party<ETC> (Nintendo) {2018.10.05} (¥5.980) - 44.867 / 1.774.663 <80-100%> (+7%)',
 '08./07. [NSW] Minecraft<ADV> (Microsoft Game Studios) {2018.06.21} (¥3.600) - 37.530 / 1.740.451 <80-100%> (-23%)',
 '09./05. [NSW] Pikmin

In [7]:
clean_tab=tab_top[2:32]
for i in range(len(clean_tab)):
    temp = clean_tab[0].split("-",1)
    
#>>> '-'.join(sentence)
#'this-is-a-sentence'

In [8]:
def flatten_list(x):
    flat_list = []
    for el in x:
        if type(el) != list:
            flat_list.append(el)
        else:
            for el2 in el:
                flat_list.append(el2)
    return flat_list

In [9]:
temp=clean_tab[0].split("/",1)
#temp[0]=float(temp[0])
temp[-1]=temp[-1].split("[",1)
temp=flatten_list(temp)
temp[-1]=temp[-1].split("]",1)
temp=flatten_list(temp)
temp

['01.',
 '01. ',
 'NSW',
 ' Momotaro Dentetsu: Showa, Heisei, Reiwa mo Teiban! <TBL> (Konami) {2020.11.19} (¥6.300) - 190.427 / 1.423.450 <80-100%> (-29%)']

In [ ]:
# https://www.neogaf.com/threads/media-create-sales-archive-thread-2.416253/#post-147729422

In [ ]:
url="https://www.resetera.com/threads/media-create-sales-archive-thread.4115/"
ustitles = requests.get(url)
soup = BeautifulSoup(ustitles.text)
links=soup.findAll("a", {"class": "link link--internal"})  #class="link link--internal" 